<a href="https://colab.research.google.com/github/biyanitanisha/PGII_TB/blob/main/Assignment_4PGII.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import ee
import geemap

In [5]:
ee.Authenticate()
ee.Initialize(project="tanisha-ee")

In [6]:
import geemap as map
map

<module 'geemap' from '/usr/local/lib/python3.12/dist-packages/geemap/__init__.py'>

In [7]:
map = geemap.Map()
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [8]:
district = ee.FeatureCollection("projects/tanisha-ee/assets/DISTRICT_BOUNDARY")

In [9]:
map.addLayer(district, {}, "District_bnd")
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [10]:
Tree_cover = ee.ImageCollection("MODIS/006/MOD44B").select("Percent_Tree_Cover")

In [11]:
tree_cover_2004 = Tree_cover.filterDate('2004-01-01', '2004-12-31').mean().clip(district)

tree_cover_2020 = Tree_cover.filterDate('2020-01-01', '2020-12-31').mean().clip(district)

params_tree2004 = {'min': 0,'max': 77,'palette': ['#f7f4f0', '#e8dcc8', '#d4c19a', '#c8e6a0', '#a8d96e', '#7cc242', '#5aaa1c', '#3d8b14', '#2d6b0f', '#1a4d0a']}
params_tree2020 = {'min': 0,'max': 77,'palette': ['#f7f4f0', '#e8dcc8', '#d4c19a', '#c8e6a0', '#a8d96e', '#7cc242', '#5aaa1c', '#3d8b14', '#2d6b0f', '#1a4d0a']}
map.addLayer(tree_cover_2004, params_tree2004, "tree_cover_2004")
map.addLayer(tree_cover_2020, params_tree2020, "tree_cover_2020")
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
def compute_zonal_stats(image, features, scale=500):
    """
    Compute zonal statistics for each district
    """
    def calc_stats(feature):
        clipped = image.clip(feature.geometry())
        stats = clipped.reduceRegion(
            reducer=ee.Reducer.mean().combine(
                ee.Reducer.min(), '', True).combine(
                ee.Reducer.max(), '', True).combine(
                ee.Reducer.stdDev(), '', True),
            geometry=feature.geometry(),
            scale=scale,
            maxPixels=1e13
        )
        return feature.set(stats)

    return features.map(calc_stats)


districts_tree_2004 = compute_zonal_stats(tree_cover_2004, district, scale=250)
districts_tree_2020 = compute_zonal_stats(tree_cover_2020, district, scale=250)

In [15]:
import geemap

geemap.ee_to_geojson(districts_tree_2004, filename='TreeCover_2004.geojson')
print("Exported: TreeCover_2004.geojson")

geemap.ee_to_geojson(districts_tree_2020, filename='TreeCover_2020.geojson')
print("Exported: TreeCover_2020.geojson")

Exported: TreeCover_2004.geojson
Exported: TreeCover_2020.geojson


In [16]:
leafareaindex = ee.ImageCollection("MODIS/061/MCD15A3H").select("Lai")

In [17]:
lai_2005 = leafareaindex.filterDate('2005-01-01', '2005-12-31').median().multiply(0.1).clip(district)
lai_2023 = leafareaindex.filterDate('2023-01-01', '2023-12-31').median().multiply(0.1).clip(district)
vis_lai02 = {'min': 0, 'max': 5.3, 'palette': ['#f7fcf5', '#e7f6e2', '#ceecc7', '#aedea7', '#88cd86', '#5db96b', '#37a055', '#1b843f', '#00682a', '#00441b']}
vis_lai23 = {'min': 0, 'max': 5.3, 'palette': ['#f7fcf5', '#e7f6e2', '#ceecc7', '#aedea7', '#88cd86', '#5db96b', '#37a055', '#1b843f', '#00682a', '#00441b']}
map.addLayer(lai_2005, vis_lai02, "lai_2005")
map.addLayer(lai_2023, vis_lai23, "lai_2020")
map

Map(bottom=812.0, center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [18]:
def compute_zonal_stats(image, features, scale=500):
    """
    Compute zonal statistics for each district
    """
    def calc_stats(feature):
        clipped = image.clip(feature.geometry())
        stats = clipped.reduceRegion(
            reducer=ee.Reducer.mean().combine(
                ee.Reducer.min(), '', True).combine(
                ee.Reducer.max(), '', True).combine(
                ee.Reducer.stdDev(), '', True),
            geometry=feature.geometry(),
            scale=scale,
            maxPixels=1e13
        )
        return feature.set(stats)

    return features.map(calc_stats)

districts_lai_2005 = compute_zonal_stats(lai_2005, district, scale=500)
districts_lai_2023 = compute_zonal_stats(lai_2023, district, scale=500)

In [19]:
import geemap

geemap.ee_to_geojson(districts_lai_2005, filename='LAI_2005.geojson')
print("Exported: LAI_2005.geojson")

geemap.ee_to_geojson(districts_lai_2023, filename='LAI_2023.geojson')
print("Exported: LAI_2023.geojson")

Exported: LAI_2005.geojson
Exported: LAI_2023.geojson


In [20]:
landsurfacetemp = ee.ImageCollection("MODIS/061/MOD11A1").select("LST_Day_1km")

In [21]:
lst_2006 = landsurfacetemp.filterDate('2006-01-01', '2006-12-31').mean().multiply(0.02).subtract(273.15).clip(district)

lst_2022 = landsurfacetemp.filterDate('2022-01-01', '2022-12-31').mean().multiply(0.02).subtract(273.15).clip(district)

vis_lst06 = {'min': 15, 'max': 45, 'palette': ['#2166ac', '#4393c3', '#92c5de', '#d1e5f0', '#fef8b8', '#fee08b', '#fdae61', '#f46d43', '#d73027', '#a50026']}
vis_lst22 = {'min': 15, 'max': 45, 'palette': ['#2166ac', '#4393c3', '#92c5de', '#d1e5f0', '#fef8b8', '#fee08b', '#fdae61', '#f46d43', '#d73027', '#a50026']}
map.addLayer(lst_2006, vis_lst06, "lst_2006")
map.addLayer(lst_2022, vis_lst22, "lst_2022")
map

Map(bottom=812.0, center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [22]:
def compute_zonal_stats(image, features, scale=500):
    """
    Compute zonal statistics for each district
    """
    def calc_stats(feature):
        clipped = image.clip(feature.geometry())
        stats = clipped.reduceRegion(
            reducer=ee.Reducer.mean().combine(
                ee.Reducer.min(), '', True).combine(
                ee.Reducer.max(), '', True).combine(
                ee.Reducer.stdDev(), '', True),
            geometry=feature.geometry(),
            scale=scale,
            maxPixels=1e13
        )
        return feature.set(stats)

    return features.map(calc_stats)

# Compute zonal statistics for LST
districts_lst_2006 = compute_zonal_stats(lst_2006, district, scale=1000)
districts_lst_2022 = compute_zonal_stats(lst_2022, district, scale=1000)

In [23]:
import geemap

geemap.ee_to_geojson(districts_lst_2006, filename='LST_2006.geojson')
print("Exported: LST_2006.geojson")

geemap.ee_to_geojson(districts_lst_2022, filename='LST_2022.geojson')
print("Exported: LST_2022.geojson")

Exported: LST_2006.geojson
Exported: LST_2022.geojson


# **Interpretation**

This study compares district-level zonal statistics derived from three MODIS datasets for two distinct time periods: the early 2000s and the mid-2020s (~2025). The variables analyzed include vegetation indices (such as NDVI), land surface temperature (LST), and related environmental indicators. By examining spatial patterns and temporal changes, the analysis highlights how vegetation cover and thermal conditions have evolved across India over the past two decades.

In the early 2000s, vegetation patterns were characterized by relatively high NDVI values in the Himalayan foothills, the northeastern states, and parts of central India where dense forests dominate. Agricultural regions in the Indo-Gangetic Plain also showed moderate vegetation cover, reflecting seasonal cropping cycles. Conversely, arid and semi-arid regions of Rajasthan and parts of Gujarat exhibited low vegetation indices, consistent with their desert and scrubland ecosystems. Land surface temperatures during this period were comparatively moderate, with cooler conditions in forested and high-altitude districts, and higher temperatures in western India’s dry zones.

By ~2025, the spatial distribution of vegetation shows notable shifts. Several districts in central and southern India display reduced NDVI values, suggesting deforestation, urban expansion, or agricultural intensification. Urban districts such as those around Delhi, Bengaluru, and Hyderabad reveal marked declines in vegetation cover, consistent with rapid urbanization. At the same time, some districts in eastern India and the northeast maintain relatively high vegetation indices, reflecting continued forest presence and higher rainfall. Interestingly, certain districts in Punjab and Haryana show fluctuating vegetation signals, likely linked to intensive agriculture and groundwater stress.

Temperature patterns reveal a clear warming trend. Districts across northern and western India exhibit higher LST values compared to the early 2000s, with urban centers showing pronounced heat signatures due to the urban heat island effect. Even traditionally cooler districts in the Himalayan region display modest increases in temperature, consistent with broader climate warming. The contrast between vegetated and non-vegetated districts has sharpened: areas with declining vegetation often correspond to rising surface temperatures, underscoring the role of vegetation in regulating local climate.

The observed changes can be attributed to multiple drivers. Urbanization has expanded rapidly, reducing green cover and increasing impervious surfaces. Agricultural intensification, particularly in northwestern India, has altered vegetation cycles and contributed to groundwater depletion, indirectly affecting vegetation health. Climate variability, including shifts in monsoon patterns, has influenced vegetation growth and temperature regimes. Deforestation and land-use change in central India further explain localized declines in vegetation.

In conclusion, the comparative analysis highlights a dual challenge: declining vegetation in several districts and rising land surface temperatures across much of India. These changes have implications for biodiversity, agriculture, and human health. However, the study is limited by the spatial resolution of MODIS data, the reliance on only two time slices, and aggregation at the district level, which may obscure finer-scale dynamics. Future work could incorporate time-series analysis, higher-resolution datasets, and integration with socio-economic variables to provide a more nuanced understanding of vegetation and temperature trends in India.